<a href="https://colab.research.google.com/github/abhi-11nav/Airline-delay-prediction/blob/main/Airline_delay_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
# Importing the libraries for dataset import and manipulation

import pandas as pd 
import numpy as np 

In [64]:
# link for raw data uploaded on github

dataset_path = "https://raw.githubusercontent.com/abhi-11nav/Airline-delay-prediction/main/Airlines.csv"

In [65]:
# Loading the dataset into the dataset variable

dataset = pd.read_csv(dataset_path)

In [5]:
# Printing the first 5 rows of the dataset

dataset.head()

NameError: ignored

In [67]:
# Printing the shape of data

dataset.shape

(539383, 9)

# DATA PREPROCESSING 

In [68]:
# Checking for missing values in the dataset 

missing_val_feature_count = 0

for col in dataset.columns:
  if dataset[col].isna().any() == True:
    missing_val_feature_count += 1

if missing_val_feature_count>0:
  print("WARNING - There are missing values in the dataset")
else:
  print("There are no missing values in the dataset")

There are no missing values in the dataset


This shows us that there are no missing values in the dataset

In [69]:
# Removing unnecessary feature "id" because it consists of all unique values

dataset.drop("id", axis=1,inplace=True)

"id" feature has been removed from the dataset

axis = 1 is to specify the axis to search on for removal of the feature,

inplace = True for removing the columns permanently from the data

.

**CLASSIFYING FEATURES AND LABELS**

DELAY IS OUR DEPENDENT VARIABLE (LABEL/Y)

REAMINING FEATURES ARE OUR INDEPENDENT VARIABLES ( FEATURES/X)

In [70]:
# Let us focus on the categorical variables of the dataset

# Printing the number of categorical variables
categorical_features = []

for col in dataset.columns:
  if type(dataset[col][0]) == str:
    categorical_features.append(col)

print(categorical_features,"are the categorical features in the dataset")

['Airline', 'AirportFrom', 'AirportTo'] are the categorical features in the dataset


In [71]:
# Now that we know how many categorical features are there in the dataset. Let us extract more information about it. 

unique_value_count = {}

for columns in categorical_features:
  unique_value_count[columns] = len(dataset[columns].value_counts())

print(unique_value_count)

{'Airline': 18, 'AirportFrom': 293, 'AirportTo': 293}


Airline feature has 18 unique values which is fine but the other two features "AirportTo" and "AirportFrom" has 293 unique values which have to be dealth with. 

In [72]:
"""# Encoding categorical variables  

#Creating columns with blank values to fill it with encoded values
for cols in categorical_features:
  dataset[cols+str("_encoded")] = 0 

for cols in categorical_features:
  for value in dataset[cols].unique():
    len = dataset[dataset[cols]==value]["Delay"].values.size
    label_ones = np.sum(dataset[dataset[cols]==value]["Delay"].values) 
    dataset[dataset[cols]==value][cols+str("_encoded")].replace((label_ones/(len-label_ones)))
    """

'# Encoding categorical variables  \n\n#Creating columns with blank values to fill it with encoded values\nfor cols in categorical_features:\n  dataset[cols+str("_encoded")] = 0 \n\nfor cols in categorical_features:\n  for value in dataset[cols].unique():\n    len = dataset[dataset[cols]==value]["Delay"].values.size\n    label_ones = np.sum(dataset[dataset[cols]==value]["Delay"].values) \n    dataset[dataset[cols]==value][cols+str("_encoded")].replace((label_ones/(len-label_ones)))\n    '

In [73]:
dataset.head()

,Airline,Flight,AirportFrom,AirportTo,DayOfWeek,Time,Length,Delay
0,CO,269,SFO,IAH,3,15,205,1
1,US,1558,PHX,CLT,3,15,222,1
2,AA,2400,LAX,DFW,3,20,165,1
3,AA,2466,SFO,DFW,3,20,195,1
4,AS,108,ANC,SEA,3,30,202,0


In [74]:
# Defining a function to encode the values of categorical columns 

def frequency_encoder(categoorical_cols, dataset):
  for cols in categorical_features:
    frequency_map = {}
    frequency_map = dataset[cols].value_counts().to_dict()
    # Mapping the frequency to the categories in the dataset 
    dataset[cols] = dataset[cols].map(frequency_map)

In [75]:
frequency_encoder(categorical_features, dataset)

In [76]:
dataset.head()

,Airline,Flight,AirportFrom,AirportTo,DayOfWeek,Time,Length,Delay
0,21118,269,11786,15819,3,15,205,1
1,34500,1558,15557,11132,3,15,222,1
2,45656,2400,16657,22153,3,20,165,1
3,45656,2466,11786,22153,3,20,195,1
4,11471,108,1307,8042,3,30,202,0


In [77]:
# Scaling down the dataset 

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [78]:
# Let us seperate the features and labels in the dataset

X = dataset.drop("Delay",axis=1)

y = dataset["Delay"]

In [79]:
X = pd.DataFrame(scaler.fit_transform(X),columns=X.columns)

In [80]:
X.head()

,Airline,Flight,AirportFrom,AirportTo,DayOfWeek,Time,Length
0,-0.909610,-1.044258,0.131911,0.568396,-0.485552,-2.833092,1.038237
1,-0.405341,-0.420778,0.540633,0.060517,-0.485552,-2.833092,1.280689
2,0.015047,-0.013509,0.659857,1.254743,-0.485552,-2.815110,0.467761
3,0.015047,0.018415,0.131911,1.254743,-0.485552,-2.815110,0.895618
4,-1.273135,-1.122133,-1.003862,-0.274312,-0.485552,-2.779144,0.995451


Now that we have X and y . Let us implement our ANN model 

# MODEL IMPLEMENTATION

In [87]:
# Importing libraries responsible for creating the model 

import tensorflow as tf
from tensorflow import keras 

from tensorflow.keras.layers import Dense, Flatten, Input, Lambda, Dropout 
from tensorflow.keras.models import Model, Sequential 

In [122]:
# Creating the model 

model = Sequential([
  keras.layers.Dense(units = X.shape[1]),
  keras.layers.Dense(units = 800, activation="relu", kernel_initializer="he_uniform"),
  keras.layers.Dense(units = 800, activation="relu", kernel_initializer="he_uniform"),
  keras.layers.Dense(units = 800, activation="relu", kernel_initializer="he_uniform"),
  keras.layers.Dense(units = 800, activation="relu", kernel_initializer="he_uniform"),
  keras.layers.Dense(units = 800, activation="relu", kernel_initializer="he_uniform"),
  keras.layers.Dense(units = 800, activation="relu", kernel_initializer="he_uniform"),
  keras.layers.Dense(units = 1, activation="sigmoid")
])

In [123]:
optimizer = tf.optimizers.Adam(learning_rate = 0.05)

model.compile(optimizer=optimizer, loss = "binary_crossentropy", metrics= ["accuracy"])

In [124]:
model.build(input_shape = X.shape)

In [125]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            (539383, 7)               56        
                                                                 
 dense_31 (Dense)            (539383, 800)             6400      
                                                                 
 dense_32 (Dense)            (539383, 800)             640800    
                                                                 
 dense_33 (Dense)            (539383, 800)             640800    
                                                                 
 dense_34 (Dense)            (539383, 800)             640800    
                                                                 
 dense_35 (Dense)            (539383, 800)             640800    
                                                                 
 dense_36 (Dense)            (539383, 800)            

In [126]:
# Splitting into training and validation data

from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(X,y,test_size=0.2)

train_X, valid_X, train_y, valid_y = train_test_split(train_X,train_y,test_size=0.25)

In [127]:
history = model.fit(train_X, train_y, epochs = 10, validation_data=([valid_X,valid_y]))

Epoch 1/10
10114/10114 [==============================] - 330s 33ms/step - loss: 459.8309 - accuracy: 0.5461 - val_loss: 0.6874 - val_accuracy: 0.5555
Epoch 2/10
10114/10114 [==============================] - 325s 32ms/step - loss: 0.6883 - accuracy: 0.5538 - val_loss: 0.6879 - val_accuracy: 0.5555
Epoch 3/10
10114/10114 [==============================] - 316s 31ms/step - loss: 0.6881 - accuracy: 0.5539 - val_loss: 0.6871 - val_accuracy: 0.5555
Epoch 4/10
10114/10114 [==============================] - 315s 31ms/step - loss: 0.6881 - accuracy: 0.5540 - val_loss: 0.6870 - val_accuracy: 0.5555
Epoch 5/10
10114/10114 [==============================] - 317s 31ms/step - loss: 0.6881 - accuracy: 0.5538 - val_loss: 0.6878 - val_accuracy: 0.5555
Epoch 6/10
10114/10114 [==============================] - 317s 31ms/step - loss: 0.6883 - accuracy: 0.5543 - val_loss: 0.6872 - val_accuracy: 0.5555
Epoch 7/10
10114/10114 [==============================] - 325s 32ms/step - loss: 0.6881 - accuracy: 0.55

In [4]:
plt.plot(history.history["accuracy"],history.history["val_accuracy"])

NameError: ignored

In [3]:
import matplotlib.pyplot as plt 
%matplotlib inline 

import seaborn as sns 